In [15]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
tf.__version__

'2.11.0'

# 7.9 Hands-On Back Propagation Algorithm

## 7.9.1 Dataset

In [16]:
N_SAMPLES = 2000 # number of sampling points
TEST_SIZE = 0.3 # testing ratio
# Use make_moons function to generate data set
X, y = make_moons(n_samples = N_SAMPLES, noise=0.2, random_state=100)
# Split traning and testing data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)
print(X.shape, y.shape)

(2000, 2) (2000,)


In [17]:
# Make a plot
def make_plot(X, y, plot_name, file_name=None, XX=None, YY=None, preds=None, dark=False):
    if (dark):
        plt.style.use('dark_background')
    else:
        sns.set_style("whitegrid")
    plt.figure(figsize=(16,12))
    axes = plt.gca()
    axes.set(xlabel="$x_1$", ylabel="$x_2$")
    plt.title(plot_name, fontsize=30)
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    if(XX is not None and YY is not None and preds is not None):
        plt.contourf(XX, YY, preds.reshape(XX.shape), 25, alpha = 1, cmap=cm.Spectral)
        plt.contour(XX, YY, preds.reshape(XX.shape), levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    # Use color to distinguish labels
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='none')
    plt.savefig('dataset.svg')
    plt.close()
# Make distribution plot
make_plot(X, y, "Classification Dataset Visualization ")
plt.show()

## 7.9.2 Network Layer

In [18]:
class Layer:
    # Fully connected layer
    def __init__(self, n_input, n_neurons, activation=None, weights=None, bias=None):
        """
        :param int n_input: input nodes
        :param int n_neurons: output nodes
        :param str activation: activation function
        :param weights: weight vectors
        :param bias: bias vectors
        """
        # Initialize weights through Normal distribution
        self.weights = weights if weights is not None else np.random.randn(n_input, n_neurons) * np.sqrt(1 / n_neurons)
        self.bias = bias if bias is not None else np.random.rand(n_neurons) * 0.1
        self.activation = activation # activation function, e.g. ’sigmoid’
        self.last_activation = None # output of activation function o
        self.error = None
        self.delta = None
    def activate(self, x):
        # Forward propagation function
        r = np.dot(x, self.weights) + self.bias # X@W+b
        # Get output through activation function
        self.last_activation = self._apply_activation(r)
        return self.last_activation
    def _apply_activation(self, r):
        # Calculate output of activation function
        if self.activation is None:
            return r # No activation function
        # ReLU
        elif self.activation == 'relu':
            return np.maximum(r, 0)
        # tanh
        elif self.activation == 'tanh':
            return np.tanh(r)
        # sigmoid
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-r))
        return r
    def apply_activation_derivative(self, r):
        # Calculate the derivative of activation functions
        # If no activation function, derivative is 1
        if self.activation is None:
            return np.ones_like(r)
        # ReLU
        elif self.activation == 'relu':
            grad = np.array(r, copy=True)
            grad[r > 0] = 1.
            grad[r <= 0] = 0.
            return grad
        # tanh
        elif self.activation == 'tanh':
            return 1 - r ** 2
        # Sigmoid
        elif self.activation == 'sigmoid':
            return r * (1 - r)
        return r
    

## 7.9.3 Neural Network

In [19]:
class NeuralNetwork:
    # Neural Network Class
    def __init__(self):
        self._layers = [] # list of network class
    def add_layer(self, layer):
        # Add layers
        self._layers.append(layer)
    def feed_forward(self, X):
        # Forward calculation
        for layer in self._layers:
            # Loop through every layer
            X = layer.activate(X)
        return X
    def backpropagation(self, X, y, learning_rate):
        # Back propagation
        # Get result of forward calculation
        output = self.feed_forward(X)
        for i in reversed(range(len(self._layers))): 
            # reverse loop
            layer = self._layers[i] # get current layer
            # If it’s output layer
            if layer == self._layers[-1]: # output layer
                layer.error = y - output
                # calculate delta
                layer.delta = layer.error * layer.apply_activation_derivative(output)
            else: # For hidden layer
                next_layer = self._layers[i + 1]
                layer.error = np.dot(next_layer.weights, next_layer.delta)
                # Calculate delta
                layer.delta = layer.error * layer.apply_activation_derivative(layer.last_activation)
                # Update weights
                for i in range(len(self._layers)):
                    layer = self._layers[i]
                    # o_i is output of previous layer
                    o_i = np.atleast_2d(X if i == 0 else self._layers[i - 1].last_activation)
                    # Gradient descent
                    layer.weights += layer.delta * o_i.T * learning_rate
    def train(self, X_train, X_test, y_train, y_test, learning_rate, max_epochs):
        y_onehot = np.zeros((y_train.shape[0], 2))
        y_onehot[np.arange(y_train.shape[0]), y_train] = 1
        mses = []
        for i in range(max_epochs): # Train 1000 epoches
            for j in range(len(X_train)): # Train one sample per time
                self.backpropagation(X_train[j], y_onehot[j], learning_rate)
                if i % 10 == 0:
                    # Print MSE Loss
                    mse = np.mean(np.square(y_onehot - self.feed_forward(X_train)))
                    mses.append(mse)
                    print('Epoch: #%s, MSE: %f' % (i, float(mse)))
                    # Print accuracy
                    print('Accuracy: %.2f%%' % (self.accuracy(self.predict(X_test), y_test.flatten()) * 100))
        return mses

In [20]:
nn = NeuralNetwork()
nn.add_layer(Layer(2, 25, 'sigmoid')) # Hidden layer 1, 2=>25
nn.add_layer(Layer(25, 50, 'sigmoid')) # Hidden layer 2, 25=>50
nn.add_layer(Layer(50, 25, 'sigmoid')) # Hidden layer 3, 50=>25
nn.add_layer(Layer(25, 2, 'sigmoid')) # Hidden layer, 25=>2

## 7.9.4 Network Training

## 7.9.5 Network Performance

In [26]:
# After training 1000 Epochs, the accuracy rate obtained on 600 samples in the test set is

# 99.17%
# code:
nn.train(X_train, X_test, y_train, y_test, 0.01, 1000)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'